In [14]:
import pyodbc

server = 'sic-match-analysis.database.windows.net'
database = 'match-analysis'
username = 'sic-admin'
password = 'ZeilenIsLeuk!'
driver= '{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)


cursor = cnxn.cursor()

In [2]:
# TABLE 1
cursor.execute("""
CREATE TABLE regattas
(regatta varchar(50) PRIMARY KEY,
boatclass varchar(50) NULL,
courseAreaId nvarchar(40) NULL)

CREATE INDEX idx ON regattas (regatta);

""")

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'regattas' in the database. (2714) (SQLExecDirectW)")

In [ ]:
# TABLE 2
cursor.execute("""
CREATE TABLE races
(regatta varchar(50) NOT NULL,
race varchar(50) NOT NULL,
raceShort varchar(50) NOT NULL,
raceId binary(16) NOT NULL,
maxWindSpd_kts decimal(10,2) NULL,
minWindSpd_kts decimal(10,2) NULL,
avgWindDir_deg decimal(10,2) NULL,
firstlegbearing_deg decimal(10,2) NULL,
course_area_id int NULL,
nr_competitors int NULL
)

CREATE UNIQUE CLUSTERED INDEX idx_rac ON races (regatta, race);
CREATE INDEX idx_reg ON races (regatta);

""")


In [ ]:
# TABLE 3
cursor.execute("""
CREATE TABLE competitors
(regatta varchar(50) NOT NULL,
comp_id binary(16) NOT NULL,
comp_name varchar(100) NULL,
nationality varchar(3) NULL,
sailId varchar(10) NULL,
[Speed (kts) (Average)] decimal(10,2) NULL,
[Speed (kts) (Average) UPWIND] decimal(10,2) NULL,
[Speed (kts) (Average) DOWNWIND] decimal(10,2) NULL,
[Speed (kts) (Average) REACHING] decimal(10,2) NULL,
[Distance Traveled (Sum)] decimal(10,2) NULL,
[Distance Traveled (Sum) UPWIND] decimal(10,2) NULL,
[Distance Traveled (Sum) DOWNWIND] decimal(10,2) NULL,
[Distance Traveled (Sum) REACHING] decimal(10,2) NULL,
[Speed (kts) (Average) null] decimal(10,2) NULL,
[Distance Traveled (Sum) null] decimal(10,2) NULL,
overall_rank int NULL,
[Number of Maneuvers (Sum)] decimal(10,2) NULL
)

CREATE UNIQUE CLUSTERED INDEX idx ON competitors (regatta, comp_id);
""")

In [ ]:
# TABLE 4
cursor.execute("""
CREATE TABLE wind
(regatta varchar(50) NOT NULL,
race varchar(50) NOT NULL,
[timepoint-ms] bigint NOT NULL,
[lat-deg] decimal(20,10) NULL,
[lng-deg] decimal(20,10) NULL,
[speed-kts] decimal(5,2) NULL,
[trueBearing-deg] decimal(5,2) NULL,
[dampenedSpeed-kts] decimal(5,2) NULL,
[dampenedTrueBearing-deg] decimal(5,2) NULL)

CREATE UNIQUE CLUSTERED INDEX idx ON wind (regatta, race, [timepoint-ms], [lat-deg], [lng-deg]);
CREATE INDEX idx_race ON wind (regatta, race);


""")

In [1]:
# TABLE 5
cursor.execute("""
CREATE TABLE legs
(regatta varchar(50) NOT NULL,
race varchar(50) NOT NULL,
leg_nr int NOT NULL,
fromWaypointId binary(16) NULL,
toWaypointId binary(16) NULL,
[to] varchar(20) NULL,
[from] varchar(20) NULL,
upOrDownwindLeg bit NULL,
leg_nr_from_finish int NULL,
distance decimal(20,2) NULL,
correlation_tacks decimal(5,4),
correlation_side decimal(5,4),
correlation_avgSOG decimal(5,4),
correlation_traveledDistance decimal(5,4),
correlation_jibes decimal(5,4)
)


CREATE UNIQUE INDEX idx ON legs (regatta, race, leg_nr);

""")

NameError: name 'cursor' is not defined

In [ ]:
# TABLE 6
cursor.execute("""
CREATE TABLE race_comp
(regatta varchar(50) NOT NULL,
race varchar(50) NOT NULL,
comp_id binary(16) NOT NULL,
rank int NULL)

CREATE UNIQUE CLUSTERED INDEX idx ON race_comp (regatta, race, comp_id);
CREATE INDEX idx_race ON race_comp (regatta, race);
CREATE INDEX idx_comp ON race_comp (regatta, comp_id);

""")

In [ ]:
# TABLE 7
cursor.execute("""
CREATE TABLE comp_leg
(regatta varchar(50) NOT NULL,
race varchar(50) NOT NULL,
leg_nr int NOT NULL,
comp_id binary(16) NOT NULL,
[competitor_distanceTraveled-m] decimal(10,2) NULL,
[competitor_averageSOG-kts] decimal(5,2) NULL,
competitor_tacks int NULL,
competitor_jibes int NULL,
competitor_penaltyCircles int NULL,
competitor_rank int NULL,
[competitor_gapToLeader-s] decimal(10,2) NULL,
[competitor_gapToLeader-m] decimal(10,2) NULL,
competitor_started bit NULL,
competitor_finished bit NULL,
avg_side decimal(5,2) NULL,
most_left decimal(5,2) NULL,
most_right decimal(5,2) NULL,
rel_rank decimal(3,2) NULL
)

CREATE UNIQUE CLUSTERED INDEX idx ON comp_leg (regatta, race, leg_nr, comp_id);
CREATE INDEX idx_rac_com ON comp_leg (regatta, race, comp_id);
CREATE INDEX idx_leg ON comp_leg (regatta, race, leg_nr);

""")

In [9]:
# TABLE 8
cursor.execute("""
CREATE TABLE positions
(regatta varchar(50) NOT NULL,
race varchar(50) NOT NULL,
leg_nr int NULL,
comp_id binary(16) NOT NULL,
timepoint_ms bigint NOT NULL,
[lat-deg] decimal(20,10) NULL,
[lng-deg] decimal(20,10) NULL,
[speed-kts] decimal(10,2) NULL,
[truebearing-deg] decimal(10,2) NULL,
calculated_windSpd decimal(10,2) NULL,
calculated_windDir decimal(10,2) NULL
)

CREATE UNIQUE CLUSTERED INDEX idx ON positions (regatta, race, leg_nr, comp_id, timepoint_ms ASC);
CREATE NONCLUSTERED INDEX idx_legnr ON positions (leg_nr) 
CREATE INDEX idx_leg ON positions (regatta, race, leg_nr, comp_id);
CREATE INDEX idx_comp ON positions (regatta, race, comp_id)
""")

In [4]:
# TABLE 9
cursor.execute("""
CREATE TABLE marks
(regatta varchar(50) NOT NULL,
race varchar(50) NOT NULL,
waypointId varchar(50) NOT NULL,
lat decimal(20) NULL,
lon decimal(20) NULL,
waypointName text NULL
)



CREATE UNIQUE INDEX idx ON marks (regatta, race, waypointId);

""")



ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'marks' in the database. (2714) (SQLExecDirectW)")

In [3]:
cursor.execute("""
CREATE TABLE course_areas (
	id int,
    course_area varchar(20),
    [lat-deg] decimal(10,7),
    [lng-deg] decimal(10,7)
    );

CREATE UNIQUE INDEX idx ON course_areas (id)

INSERT INTO course_areas(id, course_area, lat, lon)
VALUES (1, 'Enoshima', 35.1756, 139.2951),
(2, 'Fujisawa',35.1583, 139.2920),
(3, 'Kamakura', 35.1744, 139.3079),
(4, 'Sagami', 35.1513, 139.3103),
(5, 'Zushi', 35.1621, 139.3239),
(6, 'Hayama', 35.1441, 139.3285)
""")

In [17]:
# Relation 1

cursor.execute("""
ALTER TABLE races
   ADD CONSTRAINT FK_races_regattas FOREIGN KEY (regatta)
      REFERENCES regattas (regatta)
      ON DELETE CASCADE
      ON UPDATE CASCADE
;
""")

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'FK_races_regattas' in the database. (2714) (SQLExecDirectW); [42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not create constraint or index. See previous errors. (1750)")

In [19]:
# Relation 2

cursor.execute("""
ALTER TABLE competitors
   ADD CONSTRAINT FK_comp_regattas FOREIGN KEY (regatta)
      REFERENCES regattas (regatta)
      ON DELETE CASCADE
      ON UPDATE CASCADE
;
""")

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'FK_comp_regattas' in the database. (2714) (SQLExecDirectW); [42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not create constraint or index. See previous errors. (1750)")

In [20]:
# Relation 3

cursor.execute("""
ALTER TABLE wind
   ADD CONSTRAINT FK_regatta_race_wind FOREIGN KEY (regatta, race)
      REFERENCES races (regatta, race)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'FK_regatta_race_wind' in the database. (2714) (SQLExecDirectW); [42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not create constraint or index. See previous errors. (1750)")

In [21]:
# Relation 4

cursor.execute("""
ALTER TABLE legs
   ADD CONSTRAINT FK_legs_races FOREIGN KEY (regatta, race)
      REFERENCES races (regatta, race)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'FK_legs_races' in the database. (2714) (SQLExecDirectW); [42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not create constraint or index. See previous errors. (1750)")

In [22]:
# Relation 5&6
cursor.execute("""
ALTER TABLE race_comp
   ADD CONSTRAINT FK_race_comp_races FOREIGN KEY (regatta, race)
      REFERENCES races (regatta, race),
    CONSTRAINT FK_race_comp_competitors FOREIGN KEY(regatta, comp_id)
      REFERENCES competitors (regatta, comp_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'FK_race_comp_races' in the database. (2714) (SQLExecDirectW); [42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not create constraint or index. See previous errors. (1750)")

In [ ]:
# Relation 7&8

cursor.execute("""
ALTER TABLE comp_leg
   ADD CONSTRAINT FK_comp_leg_race_comp FOREIGN KEY (regatta, race, comp_id)
      REFERENCES race_comp (regatta, race,  comp_id),
      CONSTRAINT FK_comp_leg_legs FOREIGN KEY (regatta, race, leg_nr)
      REFERENCES legs (regatta, race, leg_nr)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

In [11]:
# Relation 9
cursor.execute("""
ALTER TABLE positions
   ADD CONSTRAINT FK_positions_race_comp FOREIGN KEY (regatta, race, comp_id)
      REFERENCES race_comp (regatta, race, comp_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE,
   CONSTRAINT FK_positions_comp_leg FOREIGN KEY (regatta, race, leg_nr, comp_id)
     REFERENCES comp_leg (regatta, race, leg_nr, comp_id)
""")

In [ ]:
### Markpassings, to update legnr_columnn in positions

cursor.execute("""
CREATE TABLE temp_markpassings
(regatta varchar(50) NOT NULL,
race varchar(50) NOT NULL,
comp_id binar(16) NOT NULL,
leg_nr int NOT NULL,
begin_leg_ms bigint NULL,
end_leg_ms bigint NULL

)

CREATE UNIQUE INDEX idx ON temp_markpassings (regatta, race, leg_nr, comp_id);
CREATE CLUSTERED INDEX idx_2 ON temp_markpassings (regatta, race, comp_id);

""")

In [ ]:
cursor.execute("""
ALTER TABLE temp_markpassings
ADD CONSTRAINT fk_temp_markpassings_comp_leg FOREIGN KEY (regatta, race, leg_nr, comp_id)
      REFERENCES comp_leg (regatta, race, leg_nr, comp_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

In [6]:
cursor.execute("""
ALTER TABLE races
ADD CONSTRAINT fk_races_course_areas FOREIGN KEY (course_area_id)
      REFERENCES course_areas (id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

In [ ]:
### Markpassings, to update legnr_columnn in positions

cursor.execute("""
DROP TABLE temp_markpassings
CREATE TABLE temp_markpassings
(regatta varchar(50) NOT NULL,
race varchar(50) NOT NULL,
comp_id binary(16) NOT NULL,
leg_nr int NOT NULL,
begin_leg_ms bigint NULL,
end_leg_ms bigint NULL

)

CREATE UNIQUE INDEX idx ON temp_markpassings (regatta, race, leg_nr, comp_id);
CREATE CLUSTERED INDEX idx_2 ON temp_markpassings (regatta, race, comp_id);

""")

cursor.execute("""
ALTER TABLE temp_markpassings
ADD CONSTRAINT fk_temp_markpassings_comp_leg FOREIGN KEY (regatta, race, leg_nr, comp_id)
      REFERENCES comp_leg (regatta, race, leg_nr, comp_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

In [16]:
cnxn.commit()